In [1]:
# Import necessary libraries
import pandas as pd
import glob
from pandas_profiling import ProfileReport
import numpy as np
import json
import duckdb

/var/folders/p3/prw8fq7d4t350r6vj8kcn_rh0000gn/T/ipykernel_12367/3993058375.py:4: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
# create a connection to a file called 'imdb_reviews.db'
con = duckdb.connect('imdb_reviews.db')

In [3]:
con.execute("DESCRIBE").fetchdf()

,table_name,column_names,column_types,temporary
0,dummie_director,"[movie, director_1_dum, director_2_dum, direct...","[VARCHAR, UTINYINT, UTINYINT, UTINYINT, UTINYI...",False
1,dummie_writer,"[movie, writer_1_dum, writer_2_dum, writer_3_d...","[VARCHAR, UTINYINT, UTINYINT, UTINYINT, UTINYI...",False
2,imdb_review_test,"[tconst, primaryTitle, originalTitle, startYea...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
3,imdb_review_train,"[tconst, primaryTitle, originalTitle, startYea...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
4,imdb_review_validation,"[tconst, primaryTitle, originalTitle, startYea...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
5,movie_directing,"[movie, director_1, director_2, director_3, di...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
6,movie_writer,"[movie, writer_1, writer_2, writer_3, writer_4...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
7,movielens_information,"[movieId, imdbId, tmdbId, rating, year, title,...","[BIGINT, BIGINT, DOUBLE, DOUBLE, VARCHAR, VARC...",False


In [4]:
# Load train data set
df_netflix_train = con.execute("""SELECT  imdb_review_train.tconst,
                            imdb_review_train.startYear,
                            imdb_review_train.primaryTitle,
 
                           imdb_review_train.runtimeMinutes,
                           imdb_review_train.numVotes,
                           
                           netflix_data.title,
                           netflix_data.director,
                           netflix_data.cast,
                           netflix_data.country,
                           netflix_data.release_year,
                           netflix_data.rating,
                           
                           movie_writer.writer_1,
                           movie_writer.writer_2,
                           movie_writer.writer_3,
                           movie_writer.writer_4,
                           movie_writer.writer_5,
                           movie_writer.writer_6,
                           movie_writer.writer_7,
                           movie_writer.writer_8,
                           
                           movie_directing.director_1,
                           movie_directing.director_2,
                           movie_directing.director_3,
                           movie_directing.director_4,
                           movie_directing.director_5,
                           
                           imdb_review_train.label
                           
                           
                    FROM imdb_review_train
                    
                    -- Merge the netflix information
                    LEFT JOIN netflix_data
                    ON imdb_review_train.startYear = netflix_data.release_year 
                    AND imdb_review_train.primaryTitle = netflix_data.title
   
                    -- Merge the writer information
                    LEFT JOIN movie_writer
                    ON imdb_review_train.tconst = movie_writer.movie  
                    
                    -- Merge the director information
                    LEFT JOIN movie_directing
                    ON imdb_review_train.tconst = movie_directing.movie  
                    
                    
                    """).fetchdf()
df_netflix_train


CatalogException: Catalog Error: Table with name netflix_data does not exist!
Did you mean "temp.information_schema.schemata"?

In [5]:
df_netflix_train['title'].isna().sum()

NameError: name 'df_netflix_train' is not defined

In [6]:
# create a connection to a file called 'file.db'
con = duckdb.connect('imdb_reviews.db')

# create a table and load data into it
con.sql("CREATE TABLE IF NOT EXISTS netflix_training AS SELECT * FROM df_netflix_train")

con.close()


CatalogException: Catalog Error: Table with name df_netflix_train does not exist!
Did you mean "imdb_review_train"?
LINE 1: ...ISTS netflix_training AS SELECT * FROM df_netflix_train
                                                  ^